In [3]:
import pandas as pd

import gc
####

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification
## felipe


from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_lg', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
!python -m spacy download es_core_news_lg

2023-08-14 19:42:41.088769: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 19:42:42.350424: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [1]:
!python -m spacy download es_core_news_lg

%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

2023-08-14 19:40:41.255865: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 19:40:42.403258: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 7.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.0/425.0 kB 16.8 MB/s eta 

# etl

In [5]:
import pandas as pd
import sqlite3

In [6]:


query1 = "SELECT NUMERO_PQR,DESCRIPCION_PQR FROM pqr_priorizar where MEDIO_RECEPCION !=  'VIRTUAL' and unico=1 and SINTOMA_PQR ='CONCEPTOS FACTURADOS' and FECHA_CREACION >= '2023-07-31' and TECNOLOGIA ='FO'"

query2 = "(SELECT * FROM altas where ST_TIPO_SERVICIO='FTTH' AND FASE='ORDERF' AND (tipoTramite= 'VTA' OR tipoTramite=  'Sales')) AS myquery"

In [7]:

# Connect to the SQLite database file
conn = sqlite3.connect('/content/on2/pqr_priorizar.db')

# Read the SQL query into a pandas DataFrame

pqrs = pd.read_sql_query(query1, conn)

# Close the database connection
conn.close()

# Print the DataFrame
pqrs

#MDM-PQR-39191683

,NUMERO_PQR,DESCRIPCION_PQR
0,MDM-PQR-40215407,Facebook // Cliente indica que Por favor me p...
1,MDM-PQR-40223650,Apoyo WhatsApp // Línea: 6017165304 // Nomb...
2,MDM-PQR-40223796,Apoyo WhatsApp // Línea: 6012068257 // Nom...
3,MDM-PQR-40224484,WhatsApp Masivo// Es correcto quiero confirmar...
4,MDM-PQR-40224788,Linea: 6012929899 Nombre de la persona que se ...
...,...,...
2103,MDM-PQR-40372124,CC:51683047 NOMBRE:DIANA JANETH MORA SANCHEZ...
2104,MDM-PQR-40372194,Cliente MARIA ROSALBA VALENCIA ARBELAEZ Cédula...
2105,MDM-PQR-40372420,desea cambio de metodo de facturacion factura
2106,MDM-PQR-40372587,Cliente GLADYS JAIMES RICHOUX Cédula de Ciudad...


# TRASNFORMAR LAS DATOS DE TEXTO

In [8]:
sys.path.append('../librerias')

In [9]:
from clean_text import clean_text

entro2


In [10]:
pqrs = pqrs.fillna("vacio")



In [11]:
pqrs.isnull().sum()

NUMERO_PQR         0
DESCRIPCION_PQR    0
dtype: int64

In [12]:
#declare answers and questions
descripcion=pqrs.DESCRIPCION_PQR





#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]
clean_descripcion = [clean_text(des) for des in descripcion]


len(clean_descripcion)

2108

In [13]:
clean_descripcion[0]

'  cliente indica que  por favor me puedes aclarar el  pasado apareció que si se pagaba virtualmente esté  tendría un descuento de $   no sé si era sorteo o ¿cuál era la mecánica?  porque esté  el recibo llegó por el mismo valor y se pagó de manera virtual y oportunamente '

spelling

In [14]:
from tqdm import tqdm
from spelling import correct_sentence


hola


In [15]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

clean_descripcion=corrected_sentences

Processing: 100%|██████████| 2108/2108 [04:58<00:00,  7.06it/s]


lemma token stemm

In [16]:
from lemastem import data_preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['tuya', 'cabiativa', 'ozuna', 'yilson', 'mishelle', 'raphael', 'hortua', 'abdias', 'emna', 'santisteban', 'damian', 'gómez', 'almario', 'norcy', 'elizabeht', 'kewin', 'barreiro', 'jhonnattan', 'rosio', 'cordon', 'nevardo', 'laurence', 'desly', 'pardo', 'educacion', 'tiene', 'aris', 'landazuri', 'botero', 'malory', 'capador', 'janneth', 'ismelda', 'sentida', 'margelis', 'judit', 'quito', 'munevar', 'marcelina', 'yoleidy', 'katty', 'ochoa', 'jeissy', 'helmer', 'estrategia', 'fuera', 'thania', 'julia', 'brillitte', 'royer', 'conjuntos', 'sherly', 'yudi', 'felicita', 'harby', 'marili', 'noviembre', 'viveros', 'yonni', 'puello', 'greace', 'lian', 'yanguma', 'organizacion', 'virginia', 'edyd', 'sandro', 'yovanna', 'stevan', '+', 'adriano', 'flaviano', 'mayerly', 'orduña', 'yeinson', 'molinares', 'siabato', 'norelys', 'antes', 'vizcaya', 'keila', 'jhosman', 'torrente', 'christopher', 'darlin', 'joselito', 'arianne', 'ese', 'vita', 'belkys', 'brigit', 'estás', 'geraldinne', 'tibaquira', 'yova

In [17]:

clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[190]

['validar',
 'ajuste',
 'saldo',
 'validar',
 'bajo',
 'cun',
 'informar',
 'ajuste',
 'factura']

In [18]:
clean_descripcion[0]

['favor',
 'poder',
 'aclarar',
 'pasado',
 'aparecio',
 'si',
 'pagar',
 'virtualmente',
 'tendriar',
 'descuento',
 'no',
 'si',
 'sorteo',
 'mecanico',
 'recibo',
 'llego',
 'mismo',
 'pago',
 'manera',
 'oportunamente']

In [19]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


In [20]:
from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)

In [21]:
merged_df

,descripcion
0,favor poder aclarar pasado aparecio si pagar v...
1,livian numero factura llego tarifa mensual
2,numero solicitar cancelacion plan tenia contra...
3,masivo correcto querer confirmar facturar aclarar
4,actualizar electronico actual no manejar robar
...,...
2103,facturacion
2104,desear variar factura si cancelo
2105,desear metodo facturacion factura
2106,solicitar puf factura aparecer jai errado pali...



clean_descripcion = [text for text in clean_descripcion if "svas" not in text and "directv" not in text  and "winsport" not in text and "hbo" not in text  ]

len(clean_descripcion)

In [22]:
from tfidf import TFIDF
import joblib

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from joblib import dump


X = merged_df['descripcion']
X_test=merged_df['descripcion']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_binario.joblib')

#pca = PCA(n_components=585)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 1173 features
train with old features:  (2108,)
train with new features: (2108, 804)


# cargar modelo

In [24]:
from joblib import dump

import joblib


model = joblib.load('/content/PRIORIZAR/modelo/modelo_binario.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.9747337180903012, bagging_freq=4,
               feature_fraction=0.6526645750030313,
               lambda_l1=0.02873208934418634, lambda_l2=2.3446621696223424,
               learning_rate=0.0815126268370284, min_child_samples=35,
               n_estimators=88, n_jobs=-1, num_leaves=35)


In [25]:
y_pred = model.predict(X_train_new)

pqrs['prediccion_algoritmo'] = y_pred

pqrs['descripcion_clean']=merged_df['descripcion']



# Lista de palabras a filtrar
words_to_remove = ['svas', 'directv', 'paramont', 'hbo']

# Filtrar las filas del DataFrame
c_facturados_correcion = pqrs[~pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]

c_facturados_SVAS = pqrs[pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]



c_facturados_correcion


[LightGBM] [Warning] feature_fraction is set=0.6526645750030313, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6526645750030313
[LightGBM] [Warning] lambda_l2 is set=2.3446621696223424, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.3446621696223424
[LightGBM] [Warning] lambda_l1 is set=0.02873208934418634, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.02873208934418634
[LightGBM] [Warning] bagging_fraction is set=0.9747337180903012, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9747337180903012
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-40215407,Facebook // Cliente indica que Por favor me p...,1.0,favor poder aclarar pasado aparecio si pagar v...
1,MDM-PQR-40223650,Apoyo WhatsApp // Línea: 6017165304 // Nomb...,1.0,livian numero factura llego tarifa mensual
2,MDM-PQR-40223796,Apoyo WhatsApp // Línea: 6012068257 // Nom...,0.0,numero solicitar cancelacion plan tenia contra...
3,MDM-PQR-40224484,WhatsApp Masivo// Es correcto quiero confirmar...,0.0,masivo correcto querer confirmar facturar aclarar
4,MDM-PQR-40224788,Linea: 6012929899 Nombre de la persona que se ...,1.0,actualizar electronico actual no manejar robar
...,...,...,...,...
2103,MDM-PQR-40372124,CC:51683047 NOMBRE:DIANA JANETH MORA SANCHEZ...,1.0,facturacion
2104,MDM-PQR-40372194,Cliente MARIA ROSALBA VALENCIA ARBELAEZ Cédula...,1.0,desear variar factura si cancelo
2105,MDM-PQR-40372420,desea cambio de metodo de facturacion factura,1.0,desear metodo facturacion factura
2106,MDM-PQR-40372587,Cliente GLADYS JAIMES RICHOUX Cédula de Ciudad...,0.0,solicitar puf factura aparecer jai errado pali...


In [26]:
from datetime import date

today = date.today()

year, week_num, day_of_week = today.isocalendar()





In [33]:
errores = len(c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0])
total  =len(c_facturados_correcion)
sva=len(c_facturados_SVAS)
error=(errores/total)
semana=week_num-1
index='-'.join(map(str, [year,semana]))



data = {'level_0':0,
        'errores': [errores],
        'total': [total],
        'sva': [sva],
        '%_error':[error],
        'semana':[semana],
        'index':[index]}

tracking_tipificacion = pd.DataFrame(data)


#tracking_c_facturado

In [34]:
file_path = "/content/PRIORIZAR/data/precision.txt"
with open(file_path, "r") as file:
    precision = file.read()


tracking_tipificacion["accuracy"]=precision

tracking_tipificacion

,level_0,errores,total,sva,%_error,semana,index,accuracy
0,0,429,2055,53,0.208759,31,2023-31,0.8


In [35]:
tracking_tipificacion.columns

Index(['level_0', 'errores', 'total', 'sva', '%_error', 'semana', 'index',
       'accuracy'],
      dtype='object')

cargando data apra tablero

In [38]:
antiguo=pd.read_csv('/content/tracking_c_facturado.csv',sep=';')
antiguo

,level_0,errores,total,sva,%_error,semana,index,accuracy
0,1,258,1550,45,0.166400,28,2023-28,0.8
1,0,578,2686,82,0.215100,27,2023-27,0.8
2,0,593,1895,33,0.312900,29,2023-29,0.8
3,0,585,2588,72,0.226043,30,2023-30,0.8


In [39]:
cargar = pd.concat([antiguo, tracking_tipificacion], ignore_index=True)
cargar

,level_0,errores,total,sva,%_error,semana,index,accuracy
0,1,258,1550,45,0.166400,28,2023-28,0.8
1,0,578,2686,82,0.215100,27,2023-27,0.8
2,0,593,1895,33,0.312900,29,2023-29,0.8
3,0,585,2588,72,0.226043,30,2023-30,0.8
4,0,429,2055,53,0.208759,31,2023-31,0.8


In [40]:

 
# append data frame to CSV file
cargar.to_csv('/content/tracking_c_facturado.csv', sep=';', index=False, header=True)


#pd.read_csv('/content/tracking_c_facturado.csv')

#!rclone copy  /content/tracking_c_facturado.csv one:/analitica_datos/tracking_c_facturado

In [41]:
!rclone copy  /content/tracking_c_facturado.csv one:/analitica_datos/tracking_c_facturado

In [ ]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename1 = f"{root_path}c_facturados_correcion_{current_date}.csv"
filename2 = f"{root_path}c_facturados_SVAS_{current_date}.csv"


c_facturados_correcion.to_csv(filename1, index=False)

c_facturados_SVAS.to_csv(filename2, index=False)




#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

# modelo retipificacion

filter data with 0 label

In [ ]:
retipificacion = c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0.0]
retipificacion

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



X = retipificacion['descripcion_clean']
X_test=retipificacion['descripcion_clean']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_retipificacion.joblib')

#pca = PCA(n_components=361)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



In [ ]:
model = joblib.load('/content/PRIORIZAR/modelo/retipificacion.joblib')

print(model)

In [ ]:
y_pred = model.predict(X_train_new)

retipificacion["retipificacion_algoritmo"]=y_pred

In [ ]:
mapping = {0: 'BENEFICIO DE RETENCION', 1: 'INCREMENTOS DE FACTURA', 2: 'BAJA NO INGRESADA',  3: 'CANCELACION',4: 'COMPENSACION'}

# Separate features and target variable

retipificacion['tipificacion']= retipificacion['retipificacion_algoritmo'].replace(mapping)

retipificacion['index']=index


In [ ]:
file_path = "/content/PRIORIZAR/data/f1scoreretipificacion.txt"
df = pd.read_csv(file_path, sep=',', header=None)
f1 = df.values.flatten()

retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 0, "f1-score"] = f1[0]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 1, "f1-score"] = f1[1]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 2, "f1-score"] = f1[2]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 3, "f1-score"] = f1[3]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 4, "f1-score"] = f1[4]


retipificacion

In [ ]:
retipificacion=retipificacion[['index', 'tipificacion','f1-score']]
retipificacion

In [ ]:
antiguo=pd.read_csv('/content/retipificacion.csv',sep=';')
antiguo

In [ ]:
cargar = pd.concat([antiguo, retipificacion], ignore_index=True)
cargar

In [ ]:


cargar.to_csv('/content/retipificacion.csv',sep=';', index=False, header=True)
 
pd.read_csv('/content/retipificacion.csv',sep=';')

# print message

#!rclone copy  /content/retipificacion.csv one:/analitica_datos/tracking_c_facturado

In [ ]:
!rclone copy  /content/retipificacion.csv one:/analitica_datos/tracking_c_facturado

In [ ]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename3 = f"{root_path}c_facturados_retipificacion_{current_date}.csv"



retipificacion.to_csv(filename3, index=False)





#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")